仮想環境にはいる！
source .venv/bin/activate

In [24]:
#モデルのインプット
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from pathlib import Path

# ファインチューニング前のモデル
# rinna/japanese-gpt2-mediumに変更
# rinna/japanese-gpt-neox-3.6bは試してみている

# modelName = "rinna/japanese-gpt-neox-3.6b"

#ファインチューニングしたモデルをいれる
modelName = Path("./merged_models").resolve()
print("モデル:" + str(modelName))

#CUDAかMPSの確認
#コンピュータにとって最適な環境を自動的に設定する
if torch.cuda.is_available():
    print("CUDA is available")
    device = torch.device("cuda")
    dtype = torch.float16
    device_name = torch.cuda.get_device_name(0)
    
elif torch.backends.mps.is_available():
    print("MPS is available")
    device = torch.device("mps")
    dtype = torch.float32
    device_name = "Apple Silicon (MPS)"
    
else:
    print("No CUDA or MPS support available")
    device = torch.device("cpu")
    dtype = torch.float32
    device_name = "CPU"
    
print(f"使用デバイス: {device}（{device_name}）")

# モデルとトークナイザーの読み込み
tokenizer = AutoTokenizer.from_pretrained(
    modelName,
    padding_side = 'left',
    use_fast = False,
    local_files_only=True,
     trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    modelName,
    torch_dtype = dtype,
    local_files_only=True,
    trust_remote_code=True
).to(device)

print("モデルのロード完了")

モデル:/Users/kou_fkt/Documents/GitHub/25GE/merged_models
MPS is available
使用デバイス: mps（Apple Silicon (MPS)）
モデルのロード完了


In [22]:
#jsonファイルに対話を保存する
import json
from pathlib import Path

def saveJSON (input, output, path = "chatLog/chatLog.json"):
    file = Path(path)
    
    if file.exists():
        with open(file, "r", encoding="utf-8") as f:
            chatLog = json.load(f)
            
    else:
        chatLog = []
        
    newEntry = {
        "instruction": input,
        "output": output
    }
    
    chatLog.append(newEntry)
    
    with open(file, "w", encoding="utf-8") as f:
        json.dump(chatLog, f, ensure_ascii=False, indent=2)
        
    print("対話をセーブしました")
    

In [27]:
#ユーザーに対する質問を作成
prompt = "私に対して短い質問をしてください。"

inputs = tokenizer(
    prompt,
    return_tensors="pt",
    truncation=True,
    max_length=50,
    add_special_tokens=True,
).to(device)

model.eval()

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,
        temperature=0.7,
    )

full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
query = full_text[len(prompt):]

#ユーザーに質問を投げる
prompt = str(input(query))

inputs = tokenizer(
    prompt,
    return_tensors="pt",
    truncation=True,
    max_length=512,
    add_special_tokens=True,
).to(device)

model.eval()

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,
        temperature=0.7,
    )
    
full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

generated = full_text[len(prompt):]

# 結果の出力
print("Question: " + query)
print("Input: " + prompt)
print("Output: " + generated.strip())

#対話をjsonに保存
saveJSON(query, prompt)



Question: 質問の内容は、私があなたをどう思っているのか、質問の内容は、あなたのことをどう思っているのか、あなたの気持ちをどう思っているのか、です。 あなたの気持ちをわかりやすく言い表してください。 質問の
Input: よくわかんない
Output: だってそれ、俺だったら「なにか」ってのは「なにか」じゃなくて「なにか」って判断すると思うんだよね。 でも、お前の脳内では「なにか」を「な
対話をセーブしました


In [36]:
#ファインチューニング
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
import time
from datetime import datetime
    
current_date = datetime.now().date()

while True:
    # 1秒ごとにチェック
    time.sleep(1)
    
    # 現在の日付を再取得
    new_date = datetime.now().date()
    
    # 日付が変わったら処理を実行
    if new_date != current_date:
        startFinetuning()
        current_date = new_date  # 日付を更新


def startFinetuning():
    #LoRAの設定
    peftConfig = LoraConfig(
        r = 32,
        lora_alpha = 32,
        task_type = TaskType.CAUSAL_LM,
        lora_dropout = 0.1,
        bias = "none",
    )
    model = get_peft_model(model, peftConfig)

    datasets = load_dataset("json", data_files="chatLog/chatLog.json")["train"]

    def tokenize(examples):
        return tokenizer(
            examples["instruction"],
            text_target=examples["output"],
            truncation=True,
            max_length=512,
            padding="max_length",
        )
        
    trainArgs = TrainingArguments(
        output_dir="./TunedModel",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 10,
        fp16 = True if device.type == "cuda" else False,
        save_total_limit = 2,
        logging_steps = 10,
        learning_rate = 5e-5
    )

    trainer = Trainer(
        model = model,
        args = trainArgs,
        train_dataset=datasets.map(tokenize, batched=True, remove_columns=["instruction", "output"]),
        tokenizer = tokenizer,
        data_collator = None
    )

    trainer.train()

    #モデルの保存
    model.save_pretrained("./tunedModel")
    tokenizer.save_pretrained("./tunedModel")


KeyboardInterrupt: 